In [1]:
from eunjeon import Mecab
tagger = Mecab()

In [2]:
from gensim import models
import fasttext
import fasttext.util
from gensim.models import KeyedVectors

In [3]:
review_only_model = models.fasttext.load_facebook_model("D:/embedding/word-embeddings/word-embeddings/fasttext/review_only.bin")

# 벡터 유사도 확인 방법

In [ ]:
similar_list = review_only_model.most_similar('감자튀김', topn= 30)
similar_list

# 검색어 토큰 생성 함수

In [4]:
stop_tags = ['JKS','JKC','JKG','JKO','JKB','JKV','JKQ','JX','JC','EP','EF','EC','ETN','ETM','XSN','XSV','XSA','SF','SE','SSO','SSC','SC','SY']
stop_words = ['한','곳','집','식당','장소','음식점']
def get_search_token(sent):
    spl_tokens = sent.split(' ')
    res = []
    for st in spl_tokens:
        pos_st = tagger.pos(st)
        mor_st = tagger.morphs(st)
        while pos_st and ( pos_st[-1][1] in stop_tags or pos_st[-1][0] in stop_words ):
            pos_st = pos_st[:-1]
            mor_st = mor_st[:-1]
        tmp = ''.join(mor_st)
        if len(tmp) > 1:
            res.append(tmp)
    return res

# 검색어 토큰과 벡터 유사도 높은 단어 추출 함수

In [5]:
def get_vector_similar_words(keywords):
    result = []
    for keyword in keywords:
        similar_list = review_only_model.most_similar(keyword, topn= 300)
        words = [keyword]
        res = []
        for item in similar_list:
            if keyword not in str(item[0]):
                res.append(item)
        while len(words) < 5 and res:
            if not res:
                break
            tmp = get_search_token(res.pop(0)[0])
            if tmp:
                tmp = tmp[0]
                words.append(tmp)
                tmp_res = []
                for item in res:
                    if tmp not in str(item[0]) and item[1] > 0.7:
                        tmp_res.append(item)
                res = tmp_res
        result.append(words)
    return result

# 쿼리 생성 함수

In [6]:
def get_query(tokens,table_name):
    sql = 'SELECT * FROM `'+table_name+'` WHERE {};'
    sub = ''
    for i,token_list in enumerate(tokens):
        if i != 0:
            sub += ' AND '
        sub += '({})'
        form = '(`REV_COMMENT` LIKE \'%{}%\')'
        sub2 = ''
        for j,token in enumerate(token_list):
            if j != 0:
                sub2 += ' OR '
            sub2 += form.format(token)
        sub = sub.format(sub2)
    return sql.format(sub) 

# DB 설정

In [7]:
import pymysql
hot6_db = pymysql.connect(
    user = 'duduzi',
    passwd = 'Airf1@wDB',
    host = '115.85.182.213',
    db = 'hot6',
    charset = 'utf8mb4'
)
cursor = hot6_db.cursor(pymysql.cursors.DictCursor)

# 검색방법 -> search_sentence 에 검색 문장 넣기

In [8]:
search_sentence = '국물이 진한 쌀국수집'
tokens = get_search_token(search_sentence)
print(tokens)
token_vec = get_vector_similar_words(tokens)
print(token_vec)
sql = get_query(token_vec,'TA_REVIEW')
cursor.execute(sql)
result = cursor.fetchall()
for res in result:
    print('===========================')
    print(res['REV_COMMENT'])

['국물', '진한', '쌀국수']


c:\users\vmzbn\appdata\local\programs\python\python36\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `most_similar` (Method will be removed in 4.0.0, use self.wv.most_similar() instead).
  after removing the cwd from sys.path.


[['국물', '얼큰'], ['진한', '진하', '찐한', '진해'], ['쌀국수', '분짜', '베트남', '반미', '팟타이']]
2번 방문했는데 너무 맛있습니다
국물도 진하고 한국식으로 잘 표현한 쌀국수입니다
고기빼고는 모두 리필 가능하니 양 많으신분들 리필해달라고 부탁해보세용ㅋ
보통 베트남 쌀국수가 기름진 육수와 진한 맛이 있어야 한다고 합니다.
그래서 이곳이 한국식인(?) 베트남 쌀국수를 먹을 수 있다고 들었네요. 담백하고 올라간 고추고명 때문에 속이 약간 아릴 정도로 맵습니다. (제가 매운 걸 못먹어서 그러니, 못드시는 분들은 제거하고 드셔야 할 듯 합니다.)
사실 생기고 나서부터 자주 지나갔는데, 지나갈 때마다 준비 시간이나 웨이팅 때문에 못갔습니다. 예정없이 근처에 있을 시간이 되어서 산책하다가 5시에 맞춰 갔는데... 이미 10분이 넘게 줄 서있네요. 어설프게 가시는 것은 웨이팅 때문에 도움이 안될 것 같아요. 식권을 가게 밖에서 주문해서 먹는 방식이며 직원들은 친절합니다. 가게도 바쁘고 좁은데 깨끗한 편이고요.
워낙 여행을 많이 다니다보니 이 국수가 베트남 쌀국수라 생각하긴 힘듭니다만, 오리지널보다 깔끔하고 얼큰한 느낌의 베트남식 쌀국수를 드시고 싶다면 추천합니다.
숙주랑 고기는 따로 손질해서 준비하고 육수 위에 얹는 방식으로 음식을 내 놓네요.
신촌 현대백화점에서 동교동 가는 근처인데, 부근 공원 화장실 앞에서 2호점 공사를 하고 있더군요. 부탄추 홍대점 있는 근처입니다. 웨이팅이 좀 줄어들면 편하게 식사하실 수 있을 것 같네요.
워낙에 태국이나 베트남 쌀국수를 좋아해서 그 맛을 잊지 못하는데요. 소이연남은 언제 가도 만족을 주는 곳입니다. 소고기 쌀국수와 소이뽀삐아를 함께 먹으면 동남아 부럽지 않습니다. 쌀국수의 국물 맛이 진하고, 위에 올려진 고기 고명이 너무 부드럽고 맛있어요. 약간 간이 세다고 느낄 수 있지만 소이뽀삐아와 함께 먹으면 또 밸런스가 맞는 것도 같구요. 쌀국수 면은 가는 면과 중간 면을 선택할 수 있는데 늘 중간 면만 